In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

import os
import lightgbm as lgb
from sklearn import preprocessing

In [2]:
def get_model():
    exported_pipeline = make_pipeline(
        SelectPercentile(score_func=f_classif, percentile=48),
        StackingEstimator(estimator=SGDClassifier(alpha=0.01, eta0=0.01, fit_intercept=False, l1_ratio=0.25, learning_rate="invscaling", loss="modified_huber", penalty="elasticnet", power_t=10.0)),
        ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.6000000000000001, min_samples_leaf=1, min_samples_split=3, n_estimators=100)
    )

    set_param_recursive(exported_pipeline.steps, 'random_state', 42)
    return exported_pipeline

In [3]:
def rmsle_cv(model=None, X_train=None, y_train=None):
    seed = 42
    n_folds = 5
    kf = KFold(n_folds, shuffle=True, random_state=seed).get_n_splits(X_train)
    score = cross_val_score(model, X_train, y_train,
                           scoring="f1_macro", cv = kf)
    return score

In [4]:
def feature_generate_manually():
    train_path = '../data/hy_round1_train_20200102'
    test_path = '../data/hy_round1_testA_20200102'

    train_df_list = []
    for file_name in os.listdir(train_path):
        df = pd.read_csv(os.path.join(train_path, file_name))
        train_df_list.append(df)

    test_df_list = []
    for file_name in os.listdir(test_path):
        df = pd.read_csv(os.path.join(test_path, file_name))
        test_df_list.append(df)

    train_df = pd.concat(train_df_list)
    test_df = pd.concat(test_df_list)

    train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')
    test_df['time'] = pd.to_datetime(test_df['time'], format='%m%d %H:%M:%S')

    all_df = pd.concat([train_df, test_df], sort=False)

    new_df = all_df.groupby('渔船ID').agg(x_min=('x', 'min'), x_max=('x', 'max'), x_mean=('x', 'mean'), x_std=('x', 'std'), x_skew=('x', 'skew'), x_sum=('x', 'sum'),
                y_min=('y', 'min'), y_max=('y', 'max'), y_mean=('y', 'mean'), y_std=('y', 'std'), y_skew=('y', 'skew'), y_sum=('y', 'sum'),
                v_min=('速度', 'min'), v_max=('速度', 'max'), v_mean=('速度', 'mean'), v_std=('速度', 'std'), v_skew=('速度', 'skew'), v_sum=('速度', 'sum'),
                d_min=('方向', 'min'), d_max=('方向', 'max'), d_mean=('方向', 'mean'), d_std=('方向', 'std'), d_skew=('方向', 'skew'), d_sum=('方向', 'sum'))
    new_df['x_max-x_min'] = new_df['x_max'] - new_df['x_min']
    new_df['y_max-y_min'] = new_df['y_max'] - new_df['y_min']
    new_df['x_max-y_min'] = new_df['x_max'] - new_df['y_min']
    new_df['y_max-x_min'] = new_df['y_max'] - new_df['x_min']

    new_df['slope'] = new_df['y_max-y_min'] / np.where(new_df['x_max-x_min']==0, 0.001, new_df['x_max-x_min'])
    new_df['area'] = new_df['x_max-x_min'] * new_df['y_max-y_min']

    new_df['type'] = all_df.groupby('渔船ID').agg(type=('type', 'first'))['type'].values

    X_train = new_df.drop(columns=['type']).iloc[:7000]
    y_train = new_df.iloc[:7000]['type']

    X_test = new_df.drop(columns=['type']).iloc[7000:]

    return X_train, y_train, X_test


In [5]:
def feature_generate_tsfresh():
    train_df = pd.read_csv('./train.csv')
    X_train = train_df.drop(columns=['type'])
    y_train = train_df['type']

    test_df = pd.read_csv('./test.csv')
    X_test = test_df[X_train.columns]

    base_model = lgb.LGBMClassifier(n_estimators=1000, subsample=0.8)
    base_model.fit(X_train.values, y_train)

    selected_columns = X_train.columns[np.argsort(base_model.feature_importances_)[::-1][:24]]
    print(selected_columns)

    X_train = X_train[selected_columns]
    X_test = X_test[selected_columns]

    X_train_manully, _, X_test_manully = feature_generate_manually()

    X_train['x_max-x_min'] = X_train_manully['x_max-x_min'].values
    X_test['x_max-x_min'] = X_test_manully['x_max-x_min'].values
    X_train['x_max-y_min'] = X_train_manully['x_max-y_min'].values
    X_test['x_max-y_min'] = X_test_manully['x_max-y_min'].values
    X_train['y_max-x_min'] = X_train_manully['y_max-x_min'].values
    X_test['y_max-x_min'] = X_test_manully['y_max-x_min'].values
    X_train['y_max-y_min'] = X_train_manully['y_max-y_min'].values
    X_test['y_max-y_min'] = X_test_manully['y_max-y_min'].values

    X_train['slope'] = X_train_manully['slope'].values
    X_test['slope'] = X_test_manully['slope'].values
    X_train['area'] = X_train_manully['area'].values
    X_test['area'] = X_test_manully['area'].values
    
    for column in list(X_test.columns[X_test.isnull().sum() > 0]):
        mean_val = X_test[column].mean()
        X_test[column].fillna(mean_val, inplace=True)

    return X_train.values, y_train.values, X_test.values

In [6]:
# X_train_manually, y_train, X_test_manually, feature_manually = feature_generate_manually()

In [7]:
X_train_tsfresh, y_train, X_test_tsfresh = feature_generate_tsfresh()

Index(['x__quantile__q_0.1', 'x__minimum', 'y__quantile__q_0.9', 'y__maximum',
       'x__quantile__q_0.2', 'y__quantile__q_0.8', 'y__quantile__q_0.7',
       '速度__number_crossing_m__m_1', 'y__minimum', 'x__quantile__q_0.3',
       '速度__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'y__number_cwt_peaks__n_1', 'x__maximum', 'x__quantile__q_0.9',
       '速度__quantile__q_0.7', 'x__quantile__q_0.4',
       '方向__ar_coefficient__k_10__coeff_0', 'y__quantile__q_0.6',
       '速度__fft_coefficient__coeff_6__attr_"real"',
       '方向__fft_coefficient__coeff_64__attr_"abs"',
       '速度__ratio_beyond_r_sigma__r_2',
       'x__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_2',
       'y__fft_coefficient__coeff_6__attr_"angle"',
       '速度__agg_autocorrelation__f_agg_"mean"__maxlag_40'],
      dtype='object')


In [8]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

In [9]:
X_train = np.concatenate([X_train_tsfresh], axis=1)
X_test = np.concatenate([X_test_tsfresh], axis=1)

In [10]:
from sklearn.metrics import f1_score

def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

kf = KFold(n_splits=5, random_state=42, shuffle=True)

model_list = []
score_list = []
for train_index, test_index in kf.split(X_train):
    model = get_model()
    eval_set = (X_train[test_index], y_train[test_index])
    model.fit(X_train[train_index], y_train[train_index])
    model_list.append(model)
    score_list.append(f1_score(y_train[test_index], model.predict(X_train[test_index]), average='macro'))
    
print(score_list)
print(np.mean(score_list), np.std(score_list))

[0.8997224158144128, 0.9188629968519484, 0.9276028982457136, 0.9187203465718081, 0.8993388077641771]
0.912849493049612 0.011341524479196858


In [12]:
result_list = []
for model in model_list:
    result = model.predict_proba(X_test)
    result_list.append(result)

result = np.argmax(np.sum(np.array(result_list), axis=0) / 5, axis=1)

result = le.inverse_transform(result)
pd.DataFrame(result, index=range(7000, 9000)).to_csv('result.csv', header=None)

In [13]:
X_train.shape, X_test.shape

((7000, 30), (2000, 30))

In [50]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_path = '../data/hy_round1_train_20200102'
test_path = '../data/hy_round1_testA_20200102'

train_df_list = []
for file_name in os.listdir(train_path):
    df = pd.read_csv(os.path.join(train_path, file_name))
    train_df_list.append(df)

test_df_list = []
for file_name in os.listdir(test_path):
    df = pd.read_csv(os.path.join(test_path, file_name))
    test_df_list.append(df)

train_df = pd.concat(train_df_list)
test_df = pd.concat(test_df_list)

train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')
test_df['time'] = pd.to_datetime(test_df['time'], format='%m%d %H:%M:%S')

all_df = pd.concat([train_df, test_df], sort=False)

start_x_list = []
start_y_list = []
for ship_id, group in all_df.groupby('渔船ID'):
    group = group.sort_values(by=['time'])
    start_x = group['x'].values[0]
    start_y = group['y'].values[0]

    start_x_list.append(start_x)
    start_y_list.append(start_y)

In [51]:
data_list = []
for ship_id, group in all_df.groupby('渔船ID'):
    data_list.append([])
    group = group.sort_values(by=['time'])
    hours = pd.DatetimeIndex(group['time'].values).hour
    x = group['x'].values
    y = group['y'].values
    v = group['速度'].values
    for i in range(24):
        if len(hours[hours == i]) == 0:
            data_list[-1].append(-1)
            data_list[-1].append(-1)
            data_list[-1].append(-1)
        else:
            data_list[-1].append(np.max(x[hours == i]))
            data_list[-1].append(np.max(y[hours == i]))
            data_list[-1].append(np.max(v[hours == i]))

In [67]:
from tslearn.barycenters import dtw_barycenter_averaging

data_list_2 = []
for ship_id, group in all_df.groupby('渔船ID'):
    data_list_2.append([])
    group = group.sort_values(by=['time'])
    x = group['x'].values
    y = group['y'].values
    v = group['速度'].values
    hours = pd.DatetimeIndex(group['time'].values).hour
    data_list_2[-1].append(np.mean(hours[x >= (np.max(x) * 0.9)]))
    data_list_2[-1].append(np.mean(hours[y >= (np.max(y) * 0.9)]))
    data_list_2[-1].append(np.mean(hours[v >= (np.max(v) * 0.9)]))
    data_list_2[-1] = data_list_2[-1] + dtw_barycenter_averaging(x, barycenter_size=3).flatten().tolist()
    data_list_2[-1] = data_list_2[-1] + dtw_barycenter_averaging(y, barycenter_size=3).flatten().tolist()

In [136]:
X_train = np.concatenate([X_train_tsfresh], axis=1)
X_test = np.concatenate([X_test_tsfresh], axis=1)

In [137]:
X_train = np.concatenate([X_train, np.array([start_x_list, start_y_list]).T[:7000], np.array(data_list)[:7000], np.array(data_list_2)[:7000]], axis=1)
X_test = np.concatenate([X_test, np.array([start_x_list, start_y_list]).T[7000:], np.array(data_list)[7000:], np.array(data_list_2)[7000:]], axis=1)

In [127]:
X_train.shape, X_test.shape, y_train.shape

((7000, 113), (2000, 113), (7000,))

In [128]:
from feature_selector import FeatureSelector

In [129]:
fs = FeatureSelector(data=pd.DataFrame(X_train), labels=y_train)

In [130]:
fs.identify_collinear(correlation_threshold=0.98, one_hot=False)

17 features with a correlation magnitude greater than 0.98.



In [131]:
fs.identify_zero_importance(task='classification', eval_metric='multi_logloss', n_iterations=10, early_stopping=True)

zero_importance_features = fs.ops['zero_importance']

Training Gradient Boosting Model

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[338]	valid_0's multi_logloss: 0.243666
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[318]	valid_0's multi_logloss: 0.237039
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[271]	valid_0's multi_logloss: 0.247682
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[415]	valid_0's multi_logloss: 0.197492
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[390]	valid_0's multi_logloss: 0.190479
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[375]	valid_0's multi_logloss: 0.233301
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[273]	valid_0's multi_logloss: 0.221847
Trainin

In [132]:
fs.identify_low_importance(cumulative_importance=0.99)

108 features required for cumulative importance of 0.99 after one hot encoding.
5 features do not contribute to cumulative importance of 0.99.



In [133]:
train_no_missing = fs.remove(methods = ['low_importance', 'collinear'])

Removed 22 features.


In [138]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

print(X_train.shape, X_test.shape)

X_train = X_train.iloc[:, X_train.columns.isin(train_no_missing.columns)].values
X_test = X_test.iloc[:, X_test.columns.isin(train_no_missing.columns)].values

print(X_train.shape, X_test.shape)

(7000, 113) (2000, 113)
(7000, 91) (2000, 91)


In [139]:
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

def get_model():
    exported_pipeline = make_pipeline(
        RFE(estimator=ExtraTreesClassifier(criterion="entropy", max_features=0.7000000000000001, n_estimators=150), step=0.1),
        StandardScaler(),
        StackingEstimator(estimator=SGDClassifier(alpha=0.001, eta0=0.01, fit_intercept=False, l1_ratio=1.0, learning_rate="invscaling", loss="perceptron", penalty="elasticnet", power_t=0.5)),
        GradientBoostingClassifier(learning_rate=0.3, max_depth=6, max_features=0.15000000000000002, min_samples_leaf=2, min_samples_split=2, n_estimators=200, subsample=0.8500000000000001)
    )
    set_param_recursive(exported_pipeline.steps, 'random_state', 2020)
    return exported_pipeline

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

kf = KFold(n_splits=5, random_state=42, shuffle=True)

model_list = []
score_list = []
for train_index, test_index in kf.split(X_train):
    model = get_model()
    eval_set = (X_train[test_index], y_train[test_index])
    model.fit(X_train[train_index], y_train[train_index])
    model_list.append(model)
    score_list.append(f1_score(y_train[test_index], model.predict(X_train[test_index]), average='macro'))
    
print(score_list)
print(np.mean(score_list), np.std(score_list))

[0.9042908506326598, 0.9170253285761598, 0.9085831471679948, 0.9099865473669428, 0.8920118341010177]
0.9063795415689551 0.008270342159514218
